# Ted 제목 crawling package

In [1]:
import time
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x10080')

In [2]:
driver = webdriver.Chrome(options=options)
driver.get('https://www.ted.com/talks')

In [9]:
# 언어리스트 생성
langs = driver.find_element_by_css_selector('#languages').text
langs = langs.split("\n")[1:-1]

# 언어 dict 생성
lang_dict = {
    "English": "en",
    "Español": "es",
    "日本語": "ja",
    "Português brasileiro": "pt-BR",
    "中文 (繁體)": "zh",
    "한국어": "ko"
}

In [10]:
# 언어 선택
driver.find_element_by_css_selector('#languages [lang={}]'.format(lang_dict['한국어'])).click()
time.sleep(3)

In [17]:
# 컨텐츠 가져오기
contents = driver.find_elements_by_css_selector('#browse-results > div > .col')
titles = []
for content in contents:
    title = content.find_element_by_css_selector('.media > .media__message .ga-link').text
    titles.append(title)
titles[-5:]


['더 나은 미래를 위해 청년들을 어떻게 도울 것인가?',
 '역사를 다시 쓴 공주 |레오노라 네빌(Leonora Neville)',
 '의식적 투자가들이 긴장을 높여 회사의 변화를 이끌어내는 방법',
 '왜 어떤 새들은 날 수 없을까요? |길리안 깁(Gillian Gibb)',
 '더 나은 말라리아 백신개발을 위한 열쇠']

In [19]:
# URL 가져오기
contents = driver.find_elements_by_css_selector('#browse-results > div > .col')
links = []
for content in contents:
    link = content.find_element_by_css_selector('.media > .media__message .ga-link').get_attribute('href')
    links.append(link)
links[-5:]

['https://www.ted.com/talks/henrietta_fore_how_we_can_help_young_people_build_a_better_future?language=ko',
 'https://www.ted.com/talks/leonora_neville_the_princess_who_rewrote_history?language=ko',
 'https://www.ted.com/talks/vinay_shandal_how_conscious_investors_can_turn_up_the_heat_and_make_companies_change?language=ko',
 'https://www.ted.com/talks/gillian_gibb_why_can_t_some_birds_fly?language=ko',
 'https://www.ted.com/talks/faith_osier_the_key_to_a_better_malaria_vaccine?language=ko']

In [20]:
driver.quit()

# convert code to class

- 변수 선언: webdriver, 언어, 제목(list), 링크(list), 데이터 프레임
- 함수 선언
    - 사용가능 언어 출력, 언어 설정함수, 타이틀 크롤링, 링크 크롤링 / 모두 크롤링
    - csv파일로 저장, 웹 드라이버 닫기
- 함수일반:
    - getter, setter, Docs필요

In [9]:
import time
import pandas as pd
from selenium import webdriver

class MakeTed:
    """
    Ted
    ===
    params
    -----
    ted_links: driver 생성후 이동될 경로
    
    """
    
    def __init__(self, ted_link="https://www.ted.com/talks"):
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        # window size중요: click event
        options.add_argument('window-size=1920x1080')
        # 드라이버 생성이 잘 안되면 executetable_path='/usr/local/bin/chromedriver'
        self.driver = webdriver.Chrome(options=options)
        self.language = None
        self.titles = None
        self.links = None
        self.df = None
        self.driver.get(ted_link)
    
    
    def get_languages(self):
        languages_dict = {
                "English": "en",
                "Español": "es",
                "日本語": "ja",
                "Português brasileiro": "pt-BR",
                "中文 (繁體)": "zh",
                "한국어": "ko"
            }
        return languages_dict
    
    def set_language(self, lang):
        self.language = lang
        self.driver.find_element_by_css_selector('#languages [lang={}]'.format(lang)).click()
        time.sleep(3)
        
    def _links(self):    
        contents = self.driver.find_elements_by_css_selector('#browse-results > div > .col')
        links = []
        for content in contents:
            link = content.find_element_by_css_selector('.media > .media__message .ga-link').text
            links.append(link)
        self.links = links
        print("crawling links done!")
    
    def _titles(self):
        contents = self.driver.find_elements_by_css_selector('#browse-results > div > .col')
        titles = []
        for content in contents:
            title = content.find_element_by_css_selector('.media > .media__message .ga-link').text
            titles.append(title)
        self.titles = titles
        print("crawling titles done!")
        
    def crawling(self):
        self._titles()
        self._links()
        self.df = pd.DataFrame({'title': self.titles, 'link':self.links})
    
    def save_csv(self, path='ted.csv'):
        self.df.to_csv(path, index=False)
        return 'save path: {}'.format(path)
    
    def close(self):
        self.driver.quit()

In [10]:
ted = MakeTed()

In [11]:
ted.get_languages()

{'English': 'en',
 'Español': 'es',
 '日本語': 'ja',
 'Português brasileiro': 'pt-BR',
 '中文 (繁體)': 'zh',
 '한국어': 'ko'}

In [12]:
ted.set_language('ja')

In [13]:
ted.crawling()

crawling titles done!
crawling links done!


In [15]:
ted.df

,title,link
0,光は粒子？それとも波?- コルム・ケレハー,光は粒子？それとも波?- コルム・ケレハー
1,あなたの１票は大切でしょうか？アメリカの選挙人団のしくみについて ― クリスティーナ・グリーア,あなたの１票は大切でしょうか？アメリカの選挙人団のしくみについて ― クリスティーナ・グリーア
2,言葉のアイロニー（皮肉）って何だろう？ ― クリストファー・ウォーナー,言葉のアイロニー（皮肉）って何だろう？ ― クリストファー・ウォーナー
3,ゾンビの診断 脳と体について（パート１） － ティム・ヴァースタイネン&ブラッドリー・ヴォイテック,ゾンビの診断 脳と体について（パート１） － ティム・ヴァースタイネン&ブラッドリー・ヴォイテック
4,期待とは正反対の状況、アイロニー（皮肉） / クリストファー・ワーナー,期待とは正反対の状況、アイロニー（皮肉） / クリストファー・ワーナー
5,僅かな可能性: 生命が居住可能な惑星の探索 - アリエル・アンバー,僅かな可能性: 生命が居住可能な惑星の探索 - アリエル・アンバー
6,ライバル店が近くにできるのはなぜか？,ライバル店が近くにできるのはなぜか？
7,鎮痛剤はどのようにして効くのか？/ジョージ・ザイダン,鎮痛剤はどのようにして効くのか？/ジョージ・ザイダン
8,チェロを弾く私にインスピレーションを与えてくれた母,チェロを弾く私にインスピレーションを与えてくれた母
9,ウミガメのサバイバル,ウミガメのサバイバル


# make package
- setup.py

In [ ]:
from setuptools import setup, find_packages

setup(
name='dss.ted',
packages=find_packages(),
include_package_data=True,
version='0.0.1',
author='DSS',
author_email='DSS@gmail.com',
zip_safe=False
)

# ted.py for bot/ilbs

In [16]:
import Ted
import uuid

def ted_data(lang='ko'):
    ted = Ted.MakeTed()
    try:
        ted.set_language(lang)
    except:
        pass
    ted.crawling()
    
    path = "/home/ubuntu/python3/flask/bot/static/csv"
    filename = str(uuid.uuid4().hex[0:6]) + ".csv"
    filepath = "{}/{}".format(path, filename)
    ted.save_csv(filepath)
    
    ted.close()
    
    return "static/csv/{}".format(filename)

def ted_language():
    ted = Ted.MakeTed()
    languages = ted.get_languages()
    ted.close()
    return str(languages)

ModuleNotFoundError: No module named 'Ted'

# bot.py


In [ ]:
from flask import *
from libs.slack import send_slack
from libs.forecast import forecast
from libs.naver import naver_keywords
from libs.ted import ted_language, ted_data

app = Flask(__name__)

@app.route("/")
def index():
    return "running server..."

# slack outgoing webhook function
@app.route("/slack", methods=["POST"])
def slack():
    username = request.form.get("user_name")
    token = request.form.get("token")
    text = request.form.get("text")

    print(username, token, text)

    if "날씨" in text:
        weather = forecast()
        send_slack(weather)

    if "네이버" in text:
        keywords = naver_keywords()
        send_slack(keywords)
    
    if "테드언어" in text:
        languages = ted_language()
        send_slack(languages)
        
    if "테드순위" in text:
        lang = text.split("-")[1]
        download_link = ted_data(lang)
        download_link = "http://54.180.144.232/" + download_link
        send_slack("크롤링이 완료되었습니다. download_link:{}".format(download_link))        
    return Response(), 200

app.run(debug=True)